In [13]:
import pandas as pd
from numpy import array
from math import sqrt
import scipy
from scipy.optimize import Bounds
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

data_df = pd.read_csv('data/testy_data.csv', index_col=0,
                      parse_dates=True).astype(float)/100
data_df.drop(columns='SP50',inplace=True)
data_df =data_df.loc[:,['C','GOOGL','INTC','NVDA','EBAY']]
Sample_cov_matrix = data_df.cov()
var_list_annual = data_df.var()*12
mean_list_annual = data_df.mean()*12
std_list_annual = data_df.std()*sqrt(12)
std_list_annual = std_list_annual.loc[['C','GOOGL','INTC','NVDA','EBAY']]
data_df

,C,GOOGL,INTC,NVDA,EBAY
Date,,,,,
2014-02-28,0.025300,0.029366,0.018617,0.176011,0.104699
2014-03-31,-0.021180,-0.083198,0.042569,-0.025571,-0.060065
2014-04-30,0.006513,-0.041080,0.033935,0.031267,-0.061731
2014-05-30,-0.006889,0.068744,0.032405,0.033489,-0.021223
2014-06-30,-0.009880,0.022776,0.131040,-0.024210,-0.013207
2014-07-31,0.038641,-0.008757,0.096764,-0.056095,0.055334
2014-08-29,0.056021,0.004849,0.037456,0.116306,0.050539
2014-09-30,0.003291,0.010389,-0.002864,-0.051414,0.020360
2014-10-31,0.033193,-0.034908,-0.023262,0.059079,-0.072930


Minimum Variance Frontier (MVF)
minimum attainable standard deviation of annual returns for

portfolio return function
portfolio std function

In [14]:

# W_array = array(W)

def Portfolio_std(w,cov_matrix):
    from numpy import array
    from math import sqrt
    w_np = array(w)
    cov_np_array = array(cov_matrix)
    P_variance = 0
    for i in range(5):
        for j in range(5):
            P_variance += w_np[i]*w_np[j]*cov_np_array[i,j]
    return sqrt(P_variance*12)


constrains
w add up to 1

the Minimum Variance Frontier (MVF)
Constrain expected return from 0 to 30 step 5
for i in range(0,7,1):
    print(i*0.05)

minimize func = std
constrains w add up to 1
target return (0.1)- minus return (0.1) = 0
w as input
bound?

In [15]:
target_return = 0.100
constains_expected_return = {'type': 'eq',
                            'fun':lambda x:sum(mean_list_annual*x)-target_return}



opt_constraints = ({'type': 'eq',
                            'fun':lambda x:sum(x)-1}
                       # w adds up to 1
                       ,

                       # constrain target return
                       )



def portfolio_optimization(targetfunction=Portfolio_std,
                           mean_list = mean_list_annual,
                           cov_matrix=Sample_cov_matrix,
                           constraints=opt_constraints):
    # give constrains

    start_guess = pd.Series([0.0, 0.1, 0.2, 0.3, 0.4], index=['C', 'GOOGL', 'INTC', 'NVDA', 'EBAY'])

    result = scipy.optimize.minimize(\
        lambda w:targetfunction(w, cov_matrix=cov_matrix),
                                     x0=start_guess,
                                    constraints = constraints,
                                     bounds=Bounds(-1,1)

                                     )
    w_res = pd.Series(result.x,index=start_guess.index).round(4)
    weighted_return = sum(mean_list*w_res)

    print(result.message,'Portfolio Weighting is \n',w_res,
          '\nwhen weighted_return is ',round(weighted_return,2),'at standard deviation of ',
          round(result.fun,4))

In [16]:
# portfolio_optimization(targetfunction=Portfolio_std,
#                            constraints=opt_constraints)

In [23]:
def negative_complete_return(args, mean_list=mean_list_annual, cov_matrix=Sample_cov_matrix, rf=0.03):
    w=args[0:5]
    a=args[-1]
    portfolio_return = sum(mean_list * w)
    R = portfolio_return - rf
    portfolio_variance = pow(Portfolio_std(w, cov_matrix), 2)
    y = R / a * portfolio_variance
    complete_return = y * portfolio_return + (1 - y) * rf
    return 0-complete_return

In [28]:
x0_args = [0.2, 0.2, 0.2, 0.2, 0.2, 3.2]
negative_complete_return(x0_args, mean_list_annual, Sample_cov_matrix)

-0.030311821040592414

In [ ]:
print('')